In [ ]:
import asyncio
import os
import subprocess
import sys
import pandas as pd
import re
import json
import random

# --- 1. УСТАНОВКА БИБЛИОТЕК ---
def install_libs():
    try:
        import playwright
        import bs4
    except ImportError:
        print("⏳ Устанавливаем библиотеки...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "playwright", "beautifulsoup4", "pandas"])
        os.system("playwright install chromium")
        os.system("playwright install-deps")
        print("✅ Библиотеки готовы.")

install_libs()

from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

# ==========================================
# ЧАСТЬ 1: ДРАМАТИЧЕСКИЙ ТЕАТР (DRAMTEATR)
# ==========================================

async def scrape_dram_schedule(page):
    url = "https://dramteatr39.ru/afisha"
    print(f"\n🎭 [Драмтеатр] Этап 1: Сканируем расписание на {url}...")

    try:
        await page.goto(url, timeout=90000, wait_until='domcontentloaded')
        await page.wait_for_timeout(3000)

        # Прокрутка
        scrolls = 6
        for _ in range(scrolls):
            await page.mouse.wheel(0, 4000)
            await page.wait_for_timeout(random.randint(1000, 3000))

        content = await page.content()
        soup = BeautifulSoup(content, 'html.parser')

        events_list = []
        event_links = soup.find_all('a', href=re.compile(r'/spektakli/'))
        seen_identifiers = set()

        for link in event_links:
            href = link.get('href')
            full_link = f"https://dramteatr39.ru{href}" if href.startswith('/') else href

            title = link.get_text(strip=True)
            if len(title) < 2:
                continue

            container = link.find_parent(class_='affiche__container')
            card = container
            date_block = None
            if container:
                date_block = container.select_one('.affiche__date-block')
            else:
                # Fallback for unexpected markup changes.
                card = link.find_parent()
                if card:
                    date_block = card.select_one('.affiche__date-block')

            if not card:
                continue

            card_text = card.get_text(" ", strip=True).upper()

            # Дата
            date_text = "Дата не определена"
            day_text = ""
            month_text = ""
            time_text = ""

            if date_block:
                day_node = date_block.select_one('.date-list__day')
                month_node = date_block.select_one('.date-list__month')
                time_node = date_block.select_one('.date-list__time')

                if day_node:
                    day_text = day_node.get_text(strip=True)
                if month_node:
                    month_text = month_node.get_text(strip=True)
                if time_node:
                    spans = [s.get_text(strip=True) for s in time_node.find_all('span') if s.get_text(strip=True)]
                    if spans:
                        time_text = spans[-1]
                    else:
                        raw_time = time_node.get_text(" ", strip=True)
                        parts = [p for p in raw_time.split() if ":" in p]
                        if parts:
                            time_text = parts[-1]

            if day_text and month_text:
                date_text = f"{day_text} {month_text}"
                if time_text:
                    date_text = f"{date_text} {time_text}"

            # Статус
            status = "available"
            if "БИЛЕТОВ НЕТ" in card_text or "ПРОДАНО" in card_text:
                status = "sold_out"

            # Сцена
            if "ОСНОВНАЯ" in card_text:
                scene = "Основная"
            elif "СРЕДНЯЯ" in card_text:
                scene = "Средняя"
            else:
                scene = "Малая"

            event_id = f"{date_text}_{title}"
            if event_id in seen_identifiers:
                continue
            seen_identifiers.add(event_id)

            events_list.append({
                "title": title,
                "date_raw": date_text,
                "ticket_status": status,
                "scene": scene,
                "url": full_link,
                "location": "Драматический театр"
            })

        print(f"✅ [Драмтеатр] Найдено событий: {len(events_list)}")
        return events_list
    except Exception as e:
        print(f"❌ [Драмтеатр] Ошибка расписания: {e}")
        return []

async def scrape_dram_details(context, unique_urls):
    print(f"🕵️‍♂️ [Драмтеатр] Сбор деталей для {len(unique_urls)} спектаклей...")
    details_map = {}
    page = await context.new_page()

    for url in unique_urls:
        try:
            await page.goto(url, timeout=45000, wait_until='domcontentloaded')
            try:
                await page.wait_for_selector('h1', timeout=3000)
            except:
                pass

            content = await page.content()
            soup = BeautifulSoup(content, 'html.parser')
            full_text = soup.get_text(" ", strip=True).upper()

            is_pushkin = "ПУШКИНСКАЯ КАРТА" in full_text

            images = []
            for a in soup.find_all('a', href=re.compile(r'\.(jpg|jpeg|png)$', re.IGNORECASE)):
                img_url = a['href']
                if img_url.startswith('/'):
                    img_url = f"https://dramteatr39.ru{img_url}"
                if img_url not in images:
                    images.append(img_url)

            if len(images) < 2:
                for img in soup.select('div.swiper-slide img, .gallery img, .content img'):
                    src = img.get('src')
                    if src and 'logo' not in src:
                        if src.startswith('/'):
                            src = f"https://dramteatr39.ru{src}"
                        if src not in images:
                            images.append(src)

            desc_text = ""
            desc_div = soup.find('div', class_=re.compile(r'description|text-block'))
            if not desc_div:
                header = soup.find(string=re.compile("О спектакле"))
                if header and header.parent:
                    parent = header.find_parent('div')
                    if parent:
                        desc_div = parent
            if desc_div:
                desc_text = desc_div.get_text("\n", strip=True)

            creators = ""
            c_div = soup.find('div', class_=re.compile(r'creators|team'))
            if c_div:
                creators = c_div.get_text(" | ", strip=True)

            details_map[url] = {
                "pushkin_card": is_pushkin,
                "photos": images,
                "description": desc_text[:1500],
                "creators": creators[:500]
            }
        except Exception:
            details_map[url] = {}

    await page.close()
    return details_map

async def run_dramteatr(browser):
    context = await browser.new_context(viewport={'width': 1920, 'height': 1080})
    page = await context.new_page()

    schedule = await scrape_dram_schedule(page)
    if not schedule:
        await context.close()
        return []

    unique_links = list(set(x['url'] for x in schedule))
    details = await scrape_dram_details(context, unique_links)

    await context.close()

    final_data = []
    for item in schedule:
        det = details.get(item['url'], {})
        final_data.append({**item, **det})

    return final_data

# ==========================================
# ЧАСТЬ 2: МУЗЫКАЛЬНЫЙ ТЕАТР (MUZTEATR)
# ==========================================

BASE_URL_MUZ = "https://muzteatr39.ru"

async def find_muz_afisha_link(page):
    try:
        await page.goto(BASE_URL_MUZ, timeout=30000, wait_until='domcontentloaded')
        link = page.get_by_role("link", name=re.compile("Афиша", re.IGNORECASE)).first
        href = await link.get_attribute("href")
        if href:
            return f"{BASE_URL_MUZ}{href}" if href.startswith("/") else href
    except:
        pass
    return f"{BASE_URL_MUZ}/afisha"

async def scrape_muz_schedule(page, afisha_url):
    print(f"\n🎻 [Музтеатр] Этап 1: Сканируем афишу: {afisha_url}...")
    try:
        await page.goto(afisha_url, timeout=60000, wait_until='domcontentloaded')
        await page.wait_for_timeout(2000)
    except:
        pass

    # Скролл
    try:
        for _ in range(3):
            await page.mouse.wheel(0, 4000)
            await page.wait_for_timeout(1000)
    except:
        pass

    content = await page.content()
    soup = BeautifulSoup(content, 'html.parser')

    events_list = []
    unique_ids = set()
    MONTHS_MAP = {'ЯНВАР': 'ЯНВАРЯ', 'ФЕВРАЛ': 'ФЕВРАЛЯ', 'МАРТ': 'МАРТА', 'АПРЕЛ': 'АПРЕЛЯ', 'МАЙ': 'МАЯ', 'ИЮН': 'ИЮНЯ', 'ИЮЛ': 'ИЮЛЯ', 'АВГУСТ': 'АВГУСТА', 'СЕНТЯБР': 'СЕНТЯБРЯ', 'ОКТЯБР': 'ОКТЯБРЯ', 'НОЯБР': 'НОЯБРЯ', 'ДЕКАБР': 'ДЕКАБРЯ'}
    last_seen_month = "?"

    rows = soup.find_all('div', class_=re.compile(r'row\s+afisha'))
    for row in rows:
        date_col = row.find('div', class_=re.compile(r'afisha_data'))
        day_str, month_str, time_str = "?", last_seen_month, "00:00"

        if date_col:
            col_text = date_col.get_text(" ", strip=True).upper()
            day_div = date_col.find('div', class_='afisha_nom')
            if day_div:
                day_str = day_div.get_text(strip=True)
            else:
                day_match = re.search(r'^(\d{1,2})', col_text)
                if day_match:
                    day_str = day_match.group(1)

            for root, full_name in MONTHS_MAP.items():
                if root in col_text:
                    month_str = full_name
                    last_seen_month = full_name
                    break

            time_match = re.search(r'(\d{1,2}:\d{2})', col_text)
            if time_match:
                time_str = time_match.group(1).zfill(5)

        if not day_str.isdigit():
            continue
        date_final = f"{day_str} {month_str} {time_str}"

        title_div = row.find('div', class_='afisha_spek_title')
        if not title_div:
            continue

        title = ""
        link_tag = title_div.find('a')
        if link_tag:
            title = link_tag.get_text(strip=True)
            href = link_tag.get('href')
        else:
            badge = title_div.find('span', class_='badge')
            if badge:
                badge.decompose()
            title = title_div.get_text(strip=True)
            href = None

        if not href:
            continue
        full_link = f"{BASE_URL_MUZ}{href}" if href.startswith('/') else href

        badge = row.find('span', class_='badge')
        age = badge.get_text(strip=True) if badge else ""

        row_text_full = row.get_text(" ", strip=True).upper()
        status = "available"
        if "БИЛЕТОВ НЕТ" in row_text_full or "ПРОДАНО" in row_text_full:
            status = "sold_out"

        eid = f"{date_final}_{full_link}"
        if eid in unique_ids:
            continue
        unique_ids.add(eid)

        events_list.append({
            "title": title,
            "date_raw": date_final,
            "age_restriction": age,
            "ticket_status": status,
            "url": full_link,
            "location": "Музыкальный театр"
        })

    print(f"✅ [Музтеатр] Найдено событий: {len(events_list)}")
    return events_list

async def scrape_muz_details(context, unique_urls):
    print(f"🕵️‍♂️ [Музтеатр] Сбор деталей для {len(unique_urls)} событий...")
    details = {}
    page = await context.new_page()
    BAD_IMAGES = ['ulogin', 'provider', 'icon', 'logo', 'social', 'vk.com', 'ok.ru', 'blank', 'pixel']

    for url in unique_urls:
        try:
            if not url or 'http' not in url:
                continue
            try:
                await page.goto(url, timeout=30000, wait_until='domcontentloaded')
            except:
                pass

            content = await page.content()
            soup = BeautifulSoup(content, 'html.parser')

            desc_text = ""
            meta_og = soup.find("meta", property="og:description")
            if meta_og and meta_og.get("content"):
                desc_text = meta_og["content"].strip()
            if not desc_text:
                meta_desc = soup.find("meta", attrs={"name": "description"})
                if meta_desc and meta_desc.get("content"):
                    desc_text = meta_desc["content"].strip()
            if not desc_text:
                content_div = soup.find('div', class_=re.compile(r'detail-text|news-detail|item-text'))
                if content_div:
                    raw = content_div.get_text("\n", strip=True)
                    lines = [l for l in raw.split('\n') if len(l) > 30 and 'войти' not in l.lower()]
                    desc_text = "\n".join(lines)[:2000]

            images = []
            for d in soup.find_all('div', style=re.compile(r'url\(')):
                match = re.search(r'url\([\'\"]?(.*?)[\'\"]?\)', d['style'])
                if match:
                    u = match.group(1)
                    if u.startswith('/'):
                        u = f"{BASE_URL_MUZ}{u}"
                    if not any(b in u.lower() for b in BAD_IMAGES):
                        images.append(u)

            container = soup.find('div', class_=re.compile(r'container|main|content')) or soup
            for img in container.find_all('img'):
                src = img.get('src')
                if src and not src.endswith('.svg') and not any(b in src.lower() for b in BAD_IMAGES):
                    if src.startswith('/'):
                        src = f"{BASE_URL_MUZ}{src}"
                    images.append(src)

            details[url] = {
                "pushkin_card": "ПУШКИНСКАЯ КАРТА" in soup.get_text().upper(),
                "description": desc_text,
                "photos": list(set(images))
            }
        except:
            details[url] = {}

    await page.close()
    return details

async def run_muzteatr(browser):
    context = await browser.new_context(viewport={'width': 1920, 'height': 1080})
    page = await context.new_page()

    afisha_url = await find_muz_afisha_link(page)
    events = await scrape_muz_schedule(page, afisha_url)

    if not events:
        await context.close()
        return []

    urls = list(set(x['url'] for x in events))
    details = await scrape_muz_details(context, urls)

    await context.close()

    final = []
    for ev in events:
        det = details.get(ev['url'], {})
        final.append({**ev, **det})
    return final

BASE_URL_SOBOR = "https://sobor39.ru"

# --- 2. ПАРСЕР СОБОРА ---
async def scrape_sobor_strict(page):
    target_url = "https://sobor39.ru/events/concerts/night/"
    print(f"⛪ Заходим на {target_url}...")

    try:
        await page.goto(target_url, timeout=90000, wait_until='domcontentloaded')
        await page.wait_for_timeout(5000)
    except Exception as e:
        print(f"⚠️ Предупреждение загрузки: {e}. Пробуем работать с тем, что есть.")

    # --- ПРОКРУТКА ---
    print("⬇️ Начинаем прокрутку для подгрузки событий...")

    for i in range(10):
        await page.mouse.wheel(0, 5000)
        await page.wait_for_timeout(3000)

        try:
            more_btns = page.locator("text=/Показать ещ[ёе]|Загрузить/i")
            if await more_btns.count() > 0:
                if await more_btns.first.is_visible():
                    await more_btns.first.click()
                    await page.wait_for_timeout(2000)
        except:
            pass

        if i % 2 == 0:
            print(f"   ...прокрутка {i+1}/10")

    content = await page.content()
    soup = BeautifulSoup(content, 'html.parser')

    events_list = []
    unique_keys = set()

    items = soup.find_all('div', class_='list-item')
    print(f"🔍 Найдено блоков .list-item: {len(items)}")

    for item in items:
        try:
            # 1. ЗАГОЛОВОК
            title_tag = item.find('div', class_='list-t')
            if not title_tag:
                continue
            title = title_tag.get_text(strip=True)

            # 2. ДАТА (Раздельные классы)
            date_raw = "Unknown"
            day_node = item.find('div', class_='list-day')
            month_node = item.find('div', class_='list-month')
            time_node = item.find('div', class_='list-time')

            if day_node and month_node:
                d_txt = day_node.get_text(strip=True)
                m_txt = month_node.get_text(strip=True)
                t_txt = time_node.get_text(strip=True) if time_node else ""
                date_raw = f"{d_txt} {m_txt} {t_txt}".strip()

            # 3. ССЫЛКА И СТАТУС
            url = ""
            status = "available"

            other_block = item.find('div', class_='list-other')
            if other_block:
                btn = other_block.find('a', href=True)
                if btn:
                    link = btn['href']
                    url = link if link.startswith('http') else f"{BASE_URL_SOBOR}{link}"
                    btn_text = btn.get_text(strip=True).upper()
                    if "SOLD" in btn_text or "ПРОДАНО" in btn_text:
                        status = "sold_out"

            if not url:
                status = "unknown"

            # 4. КАРТИНКА
            photo_url = ""
            img_block = item.find('div', class_='list-img')
            if img_block:
                img = img_block.find('img')
                if img and img.get('src'):
                    src = img['src']
                    if 'svg' not in src:
                        photo_url = src if src.startswith('http') else f"{BASE_URL_SOBOR}{src}"

            # 5. ОПИСАНИЕ (Markdown)
            description = ""
            descr_node = item.find('div', class_='list-descr')
            if descr_node:
                # get_text(separator="\n") заменяет <br> и закрывающие теги на перенос строки
                description = descr_node.get_text(separator="\n", strip=True)

            # Дедубликация
            key = f"{date_raw}_{title}"
            if key in unique_keys:
                continue
            unique_keys.add(key)

            events_list.append({
                "title": title,
                "date_raw": date_raw,
                "ticket_status": status,
                "url": url,
                "photos": [photo_url] if photo_url else [],
                "description": description,
                "pushkin_card": False,
                "location": "Кафедральный собор"
            })
        except Exception as e:
            print(f"⚠️ Ошибка при разборе элемента: {e}")
            continue

    print(f"✅ Успешно обработано событий: {len(events_list)}")
    return events_list

async def run_sobor(browser):
    context = await browser.new_context(viewport={'width': 1920, 'height': 1080})
    page = await context.new_page()

    data = await scrape_sobor_strict(page)
    await context.close()
    return data













# ==========================================
# ЧАСТЬ 3: ТРЕТЬЯКОВСКАЯ ГАЛЕРЕЯ
# ==========================================

BASE_URL_TRETYAKOV = "https://kaliningrad.tretyakovgallery.ru"
MAX_EVENTS_TO_PROCESS = 1000  # Production limit
MAX_DATE_TIME_COMBOS = 30

MONTHS_RU = {
    "января": 1, "февраля": 2, "марта": 3, "апреля": 4, "мая": 5, "июня": 6,
    "июля": 7, "августа": 8, "сентября": 9, "октября": 10, "ноября": 11, "декабря": 12
}


async def scrape_tretyakov_events_list(page):
    """Scrape events from /events/ page, extracting detail_url and ticket_url."""
    url = f"{BASE_URL_TRETYAKOV}/events/"
    print(f"\n🖼️ [Третьяковка] Scanning: {url}")
    
    await page.goto(url, timeout=60000, wait_until='domcontentloaded')
    await page.wait_for_timeout(3000)
    
    for _ in range(3):
        await page.mouse.wheel(0, 3000)
        await page.wait_for_timeout(random.randint(1000, 1500))
    
    events = await page.evaluate("""
        () => {
            const events = [];
            const seen = new Set();
            const BASE = 'https://kaliningrad.tretyakovgallery.ru';
            
            document.querySelectorAll('.card').forEach(card => {
                const titleEl = card.querySelector('.card_title');
                if (!titleEl) return;
                
                const title = titleEl.innerText.trim();
                if (title.toUpperCase().includes('ЭКСКУРСИЯ')) return;
                
                // Get detail_url from onclick
                let detailUrl = null;
                const onclick = card.getAttribute('onclick');
                if (onclick) {
                    const match = onclick.match(/window\\.open\\(['"]([^'"]+)['"]/);
                    if (match) detailUrl = match[1];
                }
                
                // Get ticket_url
                let ticketUrl = null;
                const ticketLink = card.querySelector('a[href*="tickets"]');
                if (ticketLink) {
                    let href = ticketLink.getAttribute('href');
                    if (href.startsWith('//')) ticketUrl = 'https:' + href;
                    else ticketUrl = href;
                }
                
                if (ticketUrl && ticketUrl.includes('timepad')) return;
                
                // Get photo
                let photo = null;
                const img = card.querySelector('img.card_img');
                if (img && img.src) {
                    photo = img.src.startsWith('/') ? BASE + img.src : img.src;
                }
                
                // Get location
                const text = card.innerText.toUpperCase();
                let location = 'Третьяковка Калининград';
                if (text.includes('АТРИУМ')) location = 'Атриум';
                else if (text.includes('КИНОЗАЛ')) location = 'Кинозал';
                
                const key = title + ticketUrl;
                if (seen.has(key)) return;
                seen.add(key);
                
                if (ticketUrl) {
                    events.push({
                        title_raw: title,
                        detail_url: detailUrl,
                        ticket_url: ticketUrl,
                        photo: photo,
                        location: location
                    });
                }
            });
            return events;
        }
    """)
    
    print(f"   ✅ Found {len(events)} events")
    return events[:MAX_EVENTS_TO_PROCESS]


async def scrape_tretyakov_detail(page, detail_url):
    """Visit detail page for title and description."""
    if not detail_url:
        return {"title": None, "description": None}
    
    full_url = f"{BASE_URL_TRETYAKOV}{detail_url}" if detail_url.startswith('/') else detail_url
    print(f"   📄 Detail: {full_url}")
    
    try:
        await page.goto(full_url, timeout=30000, wait_until='domcontentloaded')
        await page.wait_for_timeout(2000)
        
        title = None
        h1 = await page.query_selector('h1')
        if h1:
            title = (await h1.inner_text()).strip()
        
        description = None
        paragraphs = await page.query_selector_all('p')
        for p in paragraphs:
            text = (await p.inner_text()).strip()
            if len(text) > 80 and not any(skip in text.lower() for skip in ['cookie', 'политик', 'hours']):
                description = text[:500]
                break
        
        return {"title": title, "description": description}
    except Exception as e:
        print(f"      ⚠️ Detail error: {e}")
        return {"title": None, "description": None}


async def scrape_tretyakov_tickets(page, ticket_url):
    """Parse ticket page for dates, times, prices. Only clicks ACTIVE dates."""
    import datetime
    
    full_url = f"{BASE_URL_TRETYAKOV}{ticket_url}" if ticket_url.startswith('/') else ticket_url
    print(f"   🎫 Tickets: {full_url[:60]}...")
    today = datetime.date.today()
    results = []
    
    try:
        await page.goto(full_url, timeout=60000, wait_until='networkidle')
        await page.wait_for_timeout(3000)
        
        # Get ACTIVE dates only
        active_dates = await page.evaluate("""
            () => {
                const items = [];
                document.querySelectorAll('div.item.active').forEach(item => {
                    const dayEl = item.querySelector('.calendarDay');
                    const monthEl = item.querySelector('.calendarMonth');
                    if (dayEl) {
                        items.push({ 
                            day: dayEl.innerText.trim(), 
                            month: monthEl ? monthEl.innerText.trim().toLowerCase() : '' 
                        });
                    }
                });
                return items;
            }
        """)
        
        print(f"      📅 ACTIVE dates: {len(active_dates)}")
        
        for d in active_dates:
            if len(results) >= MAX_DATE_TIME_COMBOS:
                break
            
            month_num = MONTHS_RU.get(d['month'], 1)
            year = today.year + (1 if today.month >= 10 and month_num < 3 else 0)
            
            try:
                date_obj = datetime.date(year, month_num, int(d['day']))
                if date_obj < today:
                    continue
            except:
                continue
            
            date_iso = date_obj.isoformat()
            date_raw = f"{d['day']} {d['month']}"
            
            # Click date
            day_val = d['day']
            await page.evaluate(f"() => {{ document.querySelectorAll('div.item.active').forEach(i => {{ const dayEl = i.querySelector('.calendarDay'); if (dayEl && dayEl.innerText.trim() === '{day_val}') i.click(); }}); }}")
            await page.wait_for_timeout(1500)
            
            # Get times
            times = await page.evaluate("""() => [...document.querySelectorAll('label.select-time-button:not(.disabled)')].map(b => b.innerText.trim().match(/^\\d{1,2}:\\d{2}$/)?.[0]).filter(Boolean)""")
            
            if not times:
                times = ['00:00']
            
            print(f"         {date_raw}: {len(times)} times")
            
            for time_str in times:
                if len(results) >= MAX_DATE_TIME_COMBOS:
                    break
                
                # Click time
                await page.evaluate(f"() => {{ document.querySelectorAll('label.select-time-button').forEach(b => {{ if (b.innerText.includes('{time_str}')) b.click(); }}); }}")
                await page.wait_for_timeout(1500)
                
                # Click sector if present
                sectors = await page.locator('text=/[Сс]ектор/').all()
                if sectors:
                    try:
                        await sectors[0].click()
                        await page.wait_for_timeout(1000)
                    except:
                        pass
                
                # Extract price
                prices = await page.evaluate("""() => [...new Set([...document.querySelectorAll('*')].map(e => e.innerText?.match(/(\\d+)\\s*₽/)?.[1]).filter(Boolean).map(Number).filter(p => p > 100))]""")
                price = min(prices) if prices else None
                status = "available" if prices else "unknown"
                
                body = await page.inner_text("body")
                if "все билеты проданы" in body.lower():
                    status = "sold_out"
                
                results.append({
                    "parsed_date": date_iso,
                    "parsed_time": time_str,
                    "date_raw": f"{date_raw} в {time_str}",
                    "ticket_price_min": price,
                    "ticket_price_max": price,
                    "ticket_status": status,
                })
        
        return results
    except Exception as e:
        print(f"      ⚠️ Ticket error: {e}")
        return []


async def run_tretyakov(browser):
    """Main parser with proven working algorithm."""
    context = await browser.new_context(viewport={'width': 1920, 'height': 1080})
    await context.route("**/*{google,yandex,metrika,analytics}*", lambda route: route.abort())
    
    list_page = await context.new_page()
    detail_page = await context.new_page()
    ticket_page = await context.new_page()

    events_raw = await scrape_tretyakov_events_list(list_page)
    if not events_raw:
        await context.close()
        return []

    all_events = []
    
    for idx, event in enumerate(events_raw):
        print(f"\n📌 [{idx+1}/{len(events_raw)}] {event['title_raw'][:50]}...")
        
        # Clean ticket_url
        raw_url = event['ticket_url']
        # 1. Remove absolute prefix if present
        if raw_url.startswith(BASE_URL_TRETYAKOV):
            raw_url = raw_url[len(BASE_URL_TRETYAKOV):]
        elif raw_url.startswith('http'):
            # External or other domain, keep as is but careful with base concatenation
            pass
            
        # 2. Remove trailing date/time components (e.g. /2026-01-07/20:00:00)
        # Regex for /YYYY-MM-DD/HH:MM:SS or /YYYY-MM-DD/HH:MM
        import re
        clean_url = re.sub(r'/\d{4}-\d{2}-\d{2}/\d{2}:\d{2}(:\d{2})?$', '', raw_url)
        clean_url = re.sub(r'/\d{4}-\d{2}-\d{2}/\d{2}:\d{2}(:\d{2})?$', '', clean_url) # Safety repeat
        
        # Use cleaned URL for processing
        print(f"   🔗 Url: {clean_url}")
        
        # Get title and description from detail page
        detail = await scrape_tretyakov_detail(detail_page, event.get('detail_url'))
        title = detail['title'] or event['title_raw']
        description = detail['description']
        
        # Get dates/times/prices from ticket page
        entries = await scrape_tretyakov_tickets(ticket_page, clean_url)
        
        photo = event['photo']
        if photo and photo.startswith('/'):
            photo = f"{BASE_URL_TRETYAKOV}{photo}"
        
        if not entries:
            target_url = clean_url
            if target_url.startswith('/'):
                target_url = f"{BASE_URL_TRETYAKOV}{target_url}"
                
            all_events.append({
                "title": title,
                "description": description,
                "date_raw": "",
                "parsed_date": None,
                "parsed_time": None,
                "ticket_status": "unknown",
                "ticket_price_min": None,
                "ticket_price_max": None,
                "url": target_url,
                "photos": [photo] if photo else [],
                "location": event['location'],
                "scene": event['location'] if event['location'] in ["Атриум", "Кинозал"] else ""
            })
        else:
            for e in entries:
                # Construct clean direct URL
                base = clean_url
                if base.startswith('/'):
                    base = f"{BASE_URL_TRETYAKOV}{base}"
                
                direct_url = f"{base}/{e['parsed_date']}/{e['parsed_time']}:00"
                all_events.append({
                    "title": title,
                    "description": description,
                    "date_raw": e['date_raw'],
                    "parsed_date": e['parsed_date'],
                    "parsed_time": e['parsed_time'],
                    "ticket_status": e['ticket_status'],
                    "ticket_price_min": e['ticket_price_min'],
                    "ticket_price_max": e['ticket_price_max'],
                    "url": direct_url,
                    "photos": [photo] if photo else [],
                    "location": event['location'],
                    "scene": event['location'] if event['location'] in ["Атриум", "Кинозал"] else ""
                })

    print(f"\n🎉 [Третьяковка] Total: {len(all_events)} event entries")
    await context.close()
    return all_events


# --- ЗАПУСК ---
async def main():
    config = {}
    target = "all"
    if os.path.exists('run_config.json'):
        config = json.load(open('run_config.json'))
        target = config.get('target_source', 'all')

    print("🚀 ЗАПУСК СБОРА ДАННЫХ...")
    print("ℹ️ [INFO] Запускаем Драмтеатр, Музтеатр, Кафедральный собор и Третьяковскую галерею.")

    results = {}

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)

        data_dram = []
        data_muz = []
        data_sobor = []
        data_tretyakov = []

        if target in ("all", "dramteatr"):
            data_dram = await run_dramteatr(browser)
            results["dramteatr"] = data_dram
            if data_dram:
                with open('dramteatr.json', 'w', encoding='utf-8') as f:
                    json.dump(data_dram, f, ensure_ascii=False, indent=4)
                pd.DataFrame(data_dram).to_csv('dramteatr.csv', index=False)
                print(f"💾 Dramteatr saved: {len(data_dram)}")

        if target in ("all", "muzteatr"):
            data_muz = await run_muzteatr(browser)
            results["muzteatr"] = data_muz
            if data_muz:
                with open('muzteatr.json', 'w', encoding='utf-8') as f:
                    json.dump(data_muz, f, ensure_ascii=False, indent=4)
                df_muz = pd.DataFrame(data_muz)
                if 'photos' in df_muz.columns:
                    df_muz['photos'] = df_muz['photos'].apply(lambda x: ' | '.join(x) if isinstance(x, list) else x)
                df_muz.to_csv('muzteatr.csv', index=False)
                print(f"💾 Muzteatr saved: {len(data_muz)}")

        if target in ("all", "sobor"):
            data_sobor = await run_sobor(browser)
            results["sobor"] = data_sobor

        if target in ("all", "tretyakov"):
            data_tretyakov = await run_tretyakov(browser)
            results["tretyakov"] = data_tretyakov

        await browser.close()

    all_data = []
    for chunk in results.values():
        if chunk:
            all_data.extend(chunk)

    if all_data:
        results["all"] = all_data

    return results

result = await main()

sobor_data = result.get("sobor", []) if result else []
tretyakov_data = result.get("tretyakov", []) if result else []
all_data = result.get("all", []) if result else []

if sobor_data:
    with open('sobor.json', 'w', encoding='utf-8') as f:
        json.dump(sobor_data, f, ensure_ascii=False, indent=4)

    df = pd.DataFrame(sobor_data)
    if 'photos' in df.columns:
        df['photo_preview'] = df['photos'].apply(lambda x: x[0] if x else "")

    # Обрезаем описание для превью в консоли
    if 'description' in df.columns:
        df['desc_preview'] = df['description'].apply(lambda x: (x[:50] + '...') if len(x) > 50 else x)

    df.drop(columns=['photos', 'desc_preview'], inplace=True, errors='ignore')
    df.to_csv('sobor.csv', index=False)

if tretyakov_data:
    with open('tretyakov.json', 'w', encoding='utf-8') as f:
        json.dump(tretyakov_data, f, ensure_ascii=False, indent=4)

    df_tretyakov = pd.DataFrame(tretyakov_data)
    if 'photos' in df_tretyakov.columns:
        df_tretyakov['photos'] = df_tretyakov['photos'].apply(lambda x: ' | '.join(x) if isinstance(x, list) else x)
    df_tretyakov.to_csv('tretyakov.csv', index=False)

if all_data:
    df_all = pd.DataFrame(all_data)
    if 'photos' in df_all.columns:
        df_all['photo_preview'] = df_all['photos'].apply(lambda x: x[0] if isinstance(x, list) and x else "")
    if 'description' in df_all.columns:
        df_all['desc_preview'] = df_all['description'].apply(lambda x: (x[:50] + '...') if len(x) > 50 else x)
    else:
        df_all['desc_preview'] = ""

    print(f"\n🎉 ИТОГ: {len(all_data)} событий.")
    pd.set_option('display.max_colwidth', 40)
    print(df_all[['date_raw', 'title', 'desc_preview']].head(10))
else:
    print("Данные не найдены.")
